In [15]:
#get Combined Df Cell
import pandas as pd

DIST_VA = pd.read_excel('data/2324_VA_DIST_DETAILS.xlsx', sheet_name=1,index_col=0)
DIST_GRAD_RATE = pd.read_excel('data/2024_DISTRICT_GRAD_RATE.xlsx', sheet_name=1,index_col=0)
DIST_RACE_DIS = pd.read_excel('data/2324_DIST_RACE_DIS.xlsx', sheet_name=1,index_col=0)
DIST_DETAIL = pd.read_excel('data/2024_District_Details.xlsx', sheet_name=1,index_col=0)
DIST_ACHEIVE = pd.read_excel('data/23-24_Achievement_District.xlsx', sheet_name=1,index_col=0)
DIST_SPEND = pd.read_excel('data/2324_DISTRICT_SPEND_PER_PUPIL.xlsx', sheet_name=1,index_col=0)

DIST_VA = DIST_VA.drop(columns = 'Watermark')
DIST_GRAD_RATE = DIST_GRAD_RATE.drop(columns=['District Name','County','Region','Watermark'])

# Clean Distric Detail DF
import pandas as pd
import numpy as np

def transform_district_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Groups a DataFrame by district ID, creating columns for total students,
    enrollment per student group, and total attendance percentage.

    Args:
        df (pd.DataFrame): The input DataFrame with district data. It is expected
                           to have an index representing the district ID and columns
                           for 'Student Group', 'Enrollment', and 'Attendance Rate'.

    Returns:
        pd.DataFrame: A new DataFrame where each row represents a unique district,
                      with columns for total enrollment, total attendance rate, and
                      enrollment numbers for each student group.
    """
    # 1. Make a copy to prevent changes to the original DataFrame
    data = df.copy()

    # Use the DataFrame's index as the unique district identifier
    if data.index.name:
        district_id_col = data.index.name
    else:
        # Fallback if the index is not named
        district_id_col = 'District ID'
        data.index.name = district_id_col

    data.reset_index(inplace=True)

    # 2. Clean the data by converting relevant columns to numeric types
    # Errors are coerced to NaN (Not a Number) for non-numeric values
    data['Enrollment'] = pd.to_numeric(data['Enrollment'], errors='coerce')
    data['Attendance Rate'] = pd.to_numeric(data['Attendance Rate'], errors='coerce')

    # 3. Pivot the table to turn unique 'Student Group' values into columns
    # We use 'first' as the aggregation function, assuming one entry per group per district
    pivoted_data = data.pivot_table(
        index=district_id_col,
        columns='Student Group',
        values='Enrollment'
    ).fillna(0).astype(int)

    # 4. Get the overall attendance rate from the 'All Students' group
    # We create a mapping of District ID to its 'Attendance Rate'
    all_students_df = data[data['Student Group'] == 'All Students'].set_index(district_id_col)
    total_attendance_map = all_students_df['Attendance Rate']

    # 5. Combine the pivoted data with the total attendance rate
    # Rename the 'All Students' column to 'Total Students' for clarity
    if 'All Students' in pivoted_data.columns:
        pivoted_data.rename(columns={'All Students': 'Total Students'}, inplace=True)

    # Add the 'Total Attendance Percent' column by mapping it from the total_attendance_map
    pivoted_data['Total Attendance Percent'] = pivoted_data.index.map(total_attendance_map)

    # 6. Reorder columns for better readability
    if 'Total Students' in pivoted_data.columns:
        # Create a list of columns with 'Total Students' and 'Total Attendance Percent' first
        student_group_cols = [col for col in pivoted_data.columns if col not in ['Total Students', 'Total Attendance Percent']]
        final_order = ['Total Students', 'Total Attendance Percent'] + student_group_cols
        pivoted_data = pivoted_data[final_order]

    # Reset index to turn the District ID from an index back into a column
    return pivoted_data.reset_index()
DIST_DETAIL = transform_district_data(DIST_DETAIL)
DIST_DETAIL.set_index('District IRN', inplace=True)

DIST_ACHEIVE = DIST_ACHEIVE.drop(columns=['District Name','County','Region','Watermark','Maximum District Performance Index Score 2023-2024'])

DIST_SPEND = DIST_SPEND.drop(columns=['District Name','County','Region','Watermark','State-Level Expenditures per Equivalent Pupil','State-Level Expenditures per Equivalent Pupil - Federal Funds','State-Level Expenditures per Equivalent Pupil - State and Local Funds'])

# Join Dataframes
combined_df = pd.merge(DIST_VA,DIST_GRAD_RATE, on='District IRN')
combined_df = pd.merge(combined_df,DIST_DETAIL, on='District IRN')
combined_df = pd.merge(combined_df,DIST_ACHEIVE, on='District IRN')
combined_df = pd.merge(combined_df,DIST_SPEND, on='District IRN')
combined_df.drop(48975,inplace=True)
combined_df[combined_df['Performance Index Score 2023-2024'] == 'NC']

#convert Numbers
combined_df['Performance Index Score 2023-2024'] = combined_df['Performance Index Score 2023-2024'].astype(float)
combined_df['Overall Effect Size'] = combined_df['Overall Effect Size'].astype(float)
combined_df['Graduation Rate Component Percent (Weighted Graduation Rate)'] = combined_df['Graduation Rate Component Percent (Weighted Graduation Rate)'].astype(float)
combined_df['Four Year Graduation Rate - Class of 2023'] = combined_df['Four Year Graduation Rate - Class of 2023'].astype(float)
combined_df['Four Year Graduation Rate Numerator - Class of 2023'] = combined_df['Four Year Graduation Rate Numerator - Class of 2023'].astype(float)
combined_df['Four Year Graduation Rate Denominator - Class of 2023'] = combined_df['Four Year Graduation Rate Denominator - Class of 2023'].astype(float)
combined_df['Five Year Graduation Rate - Class of 2022'] = combined_df['Five Year Graduation Rate - Class of 2022'].astype(float)
combined_df['Five Year Graduation Rate Numerator - Class of 2022'] = combined_df['Five Year Graduation Rate Numerator - Class of 2022'].astype(float)
combined_df['Five Year Graduation Rate Denominator - Class of 2022'] = combined_df['Five Year Graduation Rate Denominator - Class of 2022'].astype(float)

#convert Money and Stars
combined_df['Expenditures per Equivalent Pupil'] = combined_df['Expenditures per Equivalent Pupil'].str.replace(r'[$,€£]', '', regex=True).str.replace(',', '').astype(float)
combined_df['Expenditures per Equivalent Pupil - Federal Funds'] = combined_df['Expenditures per Equivalent Pupil - Federal Funds'].str.replace(r'[$,€£]', '', regex=True).str.replace(',', '').astype(float)
combined_df['Expenditures per Equivalent Pupil - State and Local Funds'] = combined_df['Expenditures per Equivalent Pupil - State and Local Funds'].str.replace(r'[$,€£]', '', regex=True).str.replace(',', '').astype(float)

#Stars

combined_df['Progress Component Star Rating'] = combined_df['Progress Component Star Rating'].str.replace(' Stars','', regex=True).str.replace(' Star','',regex=True).astype(int)
combined_df['Graduation Rate Component Rating'] = combined_df['Graduation Rate Component Rating'].str.replace(' Stars','', regex=True).str.replace(' Star','',regex=True).astype(int)
combined_df['Achievement Component Star Rating'] = combined_df['Achievement Component Star Rating'].str.replace(' Stars','', regex=True).str.replace(' Star','',regex=True).astype(int)


C:\Users\kdelv\miniconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\kdelv\miniconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [16]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
df = combined_df
numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
correlation_matrix = df[numerical_cols].corr()

# Plotting the heatmap
plt.figure(figsize=(16, 12))
sns.heatmap(correlation_matrix[['Performance Index Score 2023-2024']].sort_values(by='Performance Index Score 2023-2024', ascending=False), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation of Numerical Features with Performance Index Score 2023-2024')
plt.savefig('correlation_heatmap.png', bbox_inches='tight')
plt.close()

# --- Feature Importance with Random Forest ---
X = df[numerical_cols].drop(columns=['Performance Index Score 2023-2024'])
y = df['Performance Index Score 2023-2024']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

feature_importances = pd.DataFrame({'feature': X.columns, 'importance': rf_model.feature_importances_})
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

# Plotting feature importances
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_importances)
plt.title('Top Factors Influencing Performance Index Score 2023-2024 (Random Forest)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.savefig('feature_importance.png', bbox_inches='tight')
plt.close()

print("Top 10 features from correlation analysis:")
print(correlation_matrix['Performance Index Score 2023-2024'].abs().sort_values(ascending=False).head(11))
print("\nTop 10 features from Random Forest model:")
print(feature_importances.head(10))

Top 10 features from correlation analysis:
Performance Index Score 2023-2024      1.000000
Performance Index Percent 2023-2024    0.999992
Performance Index Score 2022-2023      0.989583
Percent of Students Limited            0.981579
Performance Index Score 2021-2022      0.979589
Achievement Component Star Rating      0.943162
Percent of Students Accomplished       0.933518
Percent of Students Advanced           0.923983
Percent of Students Basic              0.876189
Total Attendance Percent               0.841998
Graduation Rate Component Rating       0.647344
Name: Performance Index Score 2023-2024, dtype: float64

Top 10 features from Random Forest model:
                                feature  importance
25  Performance Index Percent 2023-2024    0.942423
33    Performance Index Score 2022-2023    0.022620
27          Percent of Students Limited    0.012800
24    Achievement Component Star Rating    0.007217
30     Percent of Students Accomplished    0.007025
34    Performance 

In [17]:
# Introducing some correlation for a better example

# 1. Select only the numerical columns from your DataFrame
numerical_cols = df.select_dtypes(include=np.number)

# 2. Calculate the full correlation matrix
correlation_matrix = numerical_cols.corr()

# 3. Isolate the correlations with your target variable 'Performance Index Score 2023-2024'
composite_correlation = correlation_matrix['Performance Index Score 2023-2024']

# 4. Sort the values to easily see the most positive and negative correlations
sorted_correlation = composite_correlation.sort_values(ascending=False)

# 5. Print the results
print("Correlation of each feature with 'Performance Index Score 2023-2024':")
print(sorted_correlation)

Correlation of each feature with 'Performance Index Score 2023-2024':
Performance Index Score 2023-2024                                1.000000
Performance Index Percent 2023-2024                              0.999992
Performance Index Score 2022-2023                                0.989583
Performance Index Score 2021-2022                                0.979589
Achievement Component Star Rating                                0.943162
Percent of Students Accomplished                                 0.933518
Percent of Students Advanced                                     0.923983
Total Attendance Percent                                         0.841998
Graduation Rate Component Rating                                 0.647344
Graduation Rate Component Percent (Weighted Graduation Rate)     0.647291
Four Year Graduation Rate - Class of 2023                        0.633509
Percent of Students Advanced Plus                                0.548336
Five Year Graduation Rate - Class of 2022 

In [19]:
combined_df.info()
print(combined_df)

<class 'pandas.core.frame.DataFrame'>
Index: 606 entries, 442 to 139303
Data columns (total 43 columns):
 #   Column                                                         Non-Null Count  Dtype  
---  ------                                                         --------------  -----  
 0   District Name                                                  606 non-null    object 
 1   County                                                         606 non-null    object 
 2   Region                                                         606 non-null    object 
 3   Progress Component Star Rating                                 606 non-null    int64  
 4   Overall Composite                                              606 non-null    object 
 5   Overall Effect Size                                            606 non-null    float64
 6   Graduation Rate Component Rating                               606 non-null    int64  
 7   Graduation Rate Component Percent (Weighted Graduation Rate)  